In this kernel, i join Santander - one of Kaggle Competition that use big file size dataset. The dataset size for training about 288 MB with 202 columns and for test about 287 MB with 201 columns. so i use tools dask for processing data in order to get more quickly.

### Import Library

In [1]:
#core package for data science
import numpy as np
import pandas as pd
import dask.dataframe as dd

#import package for data visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#import machine learning library stuf and others
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

#hide warnings so you can't find any warning
import warnings 
warnings.simplefilter('ignore')

print("Done !!!")

Done !!!


Display memory used for processing data using dask with memory limit is 4GB

In [2]:
from dask.distributed import Client, progress
client = Client(processes=False,threads_per_worker=4,
               n_workers=1,memory_limit='3GB')
client

Client Scheduler: inproc://192.168.0.108/3660/1 Dashboard: http://localhost:8787/status,Cluster Workers: 1 Cores: 4 Memory: 3.00 GB


### Load Dataset

#### Load dataset for training

In [3]:
%time train = dd.read_csv('data/train.csv')

Wall time: 81.8 ms


In [4]:
%time train.head()

Wall time: 1.57 s


,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


Check column size and data types

In [5]:
train.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 202 entries, ID_code to var_199
dtypes: object(1), float64(200), int64(1)

In [6]:
train.ID_code.dtype

dtype('O')

In [7]:
train.target.dtype

dtype('int64')

We get info that this dataset contain 3 kind data type: 
- 1 columns(ID_code) have data type Object.
- 200 columns(var_0 till var_199) have data type float64.
- 1 columns(target) have data type int64.

So for next step before create machine learning model, we should drop ID_code and target columns.

In [8]:
%time train.describe().compute()

Wall time: 51.5 s


,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.100490,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,...,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537
std,0.300653,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,...,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,...,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800
25%,0.000000,8.479925,-4.714825,8.743000,5.296475,9.892150,-11.133500,4.775000,13.982700,-2.292250,...,-0.014650,5.172050,0.897100,0.616000,15.664000,-1.151850,-1.823325,8.263100,13.846600,-11.083275
50%,0.000000,10.541400,-1.587050,10.611500,6.837300,11.121600,-4.789700,5.390400,16.504500,0.453350,...,3.231250,7.360900,1.907850,3.429900,17.965200,-0.154650,2.499100,8.897300,15.945500,-2.752700
75%,0.000000,12.784725,1.398400,12.550900,8.341475,12.277100,1.073825,6.012300,19.122100,2.974750,...,6.428825,9.532550,2.962800,6.259100,20.426600,0.848125,6.604900,9.599625,18.078900,4.892350
max,1.000000,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.447700,27.691800,10.151300,...,18.440900,16.716500,8.402400,18.281800,27.928800,4.272900,18.321500,12.000400,26.079100,28.500700


From this table, briefly we get insight that are :
- Count of all dataset is 200k
- ID_code columns not display maybe because this columns have 'Object' data type
- Value of target columns only consist 0 and 1
- Most value of 200 columns(var_0 till var_199) have minus value. So we need normalize this value before process into machine learning model. 

Check missing value for dataset training

In [5]:
%time train.isnull().any().compute()

Wall time: 5.96 s


ID_code    False
target     False
var_0      False
var_1      False
var_2      False
var_3      False
var_4      False
var_5      False
var_6      False
var_7      False
var_8      False
var_9      False
var_10     False
var_11     False
var_12     False
var_13     False
var_14     False
var_15     False
var_16     False
var_17     False
var_18     False
var_19     False
var_20     False
var_21     False
var_22     False
var_23     False
var_24     False
var_25     False
var_26     False
var_27     False
           ...  
var_170    False
var_171    False
var_172    False
var_173    False
var_174    False
var_175    False
var_176    False
var_177    False
var_178    False
var_179    False
var_180    False
var_181    False
var_182    False
var_183    False
var_184    False
var_185    False
var_186    False
var_187    False
var_188    False
var_189    False
var_190    False
var_191    False
var_192    False
var_193    False
var_194    False
var_195    False
var_196    False
var_197    Fal

Looks like no missing value in training dataset so we continue to load testing dataset

#### Load dataset for testing

In [5]:
%time test = dd.read_csv('data/test.csv')

Wall time: 74.8 ms


In [6]:
%time test.head()

Wall time: 1.7 s


,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [12]:
test.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 201 entries, ID_code to var_199
dtypes: object(1), float64(200)

From this info, we have 1 'Object' data type that is ID_code columns and have 200 'float64' data type that are var_0 till var_199

In [13]:
%time test.describe().compute()

Wall time: 20.8 s


,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,10.658737,-1.624244,10.707452,6.788214,11.076399,-5.050558,5.415164,16.529143,0.277135,7.569407,...,3.189766,7.458269,1.925944,3.322016,17.996967,-0.133657,2.290899,8.912428,15.869184,-3.246342
std,3.036716,4.040509,2.633888,2.052724,1.616456,7.869293,0.864686,3.424482,3.333375,1.231865,...,4.551239,3.025189,1.479966,3.995599,3.140652,1.429678,5.446346,0.920904,3.008717,10.398589
min,0.188700,-15.043400,2.355200,-0.022400,5.484400,-27.767000,2.216400,5.713700,-9.956000,4.243300,...,-14.093300,-2.407000,-3.340900,-11.413100,9.382800,-4.911900,-13.944200,6.169600,6.584000,-39.457800
25%,8.463300,-4.684200,8.764775,5.243425,9.906600,-11.126550,4.778825,13.971600,-2.280400,6.634300,...,-0.073700,5.186850,0.900300,0.640100,15.664800,-1.142700,-1.925200,8.263400,13.869550,-11.069500
50%,10.527000,-1.572750,10.568800,6.842400,11.110900,-4.746900,5.402300,16.446800,0.390700,7.635400,...,3.175600,7.396300,1.916800,3.460300,17.998700,-0.137700,2.475900,8.898200,15.952400,-2.639950
75%,12.765600,1.366200,12.511300,8.338125,12.279900,1.018775,6.010100,19.122700,2.965900,8.592800,...,6.394500,9.561800,2.973500,6.206400,20.418200,0.844900,6.539800,9.602600,18.064950,5.009400
max,22.323400,9.385100,18.714100,13.142000,16.037100,17.253700,8.302500,28.292800,9.665500,11.003600,...,20.359000,16.716500,8.005000,17.632600,27.947800,4.545400,15.920700,12.275800,26.538400,27.907400


### Drop non important features 

I drop ID_code and target columns from training dataset 

In [7]:
%time data_train = train.drop(['ID_code','target'],axis=1)

Wall time: 41.9 ms


In [8]:
%time data_train.head()

Wall time: 1.74 s


,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


I drop ID_code from testing dataset 

In [9]:
%time data_test = test.drop(['ID_code'],axis=1)

Wall time: 41.9 ms


In [10]:
%time data_test.head()

Wall time: 1.69 s


,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


### Normalize Data 

I use MinMax Scaler for normalize data

In [11]:
from sklearn.preprocessing import MinMaxScaler

In [12]:
%time train_mms = MinMaxScaler().fit_transform(data_train)

Wall time: 20.8 s


In [13]:
train_mms

array([[0.42785307, 0.32482435, 0.56805853, ..., 0.43095798, 0.32765751,
        0.56064496],
       [0.55721218, 0.42863943, 0.6812351 , ..., 0.4682771 , 0.6095461 ,
        0.60582746],
       [0.41196889, 0.48377668, 0.57806091, ..., 0.38194973, 0.42583343,
        0.58273586],
       ...,
       [0.5432771 , 0.39305749, 0.4870996 , ..., 0.45612437, 0.38087342,
        0.63919915],
       [0.46750324, 0.25309006, 0.66682332, ..., 0.67445942, 0.46661746,
        0.37047369],
       [0.5258457 , 0.36714503, 0.580254  , ..., 0.36840624, 0.29469964,
        0.57890533]])

In [14]:
%time test_mms = MinMaxScaler().fit_transform(data_test)

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.29 GB -- Worker memory limit: 3.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.36 GB -- Worker memory limit: 3.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.39 GB -- Worker memory limit: 3.00 GB
distributed.worker - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 2.41 GB -- Worker memory limit: 3.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.41 GB -- Worker memory limit: 3.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to 

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.54 GB -- Worker memory limit: 3.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.56 GB -- Worker memory limit: 3.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.57 GB -- Worker memory limit: 3.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.59 GB -- Worker memory limit: 3.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.61 GB -- Worker memory limit: 3.00 GB
distributed.worker - WARN

Wall time: 32.4 s


In [15]:
test_mms

array([[0.49139586, 0.93428577, 0.64786752, ..., 0.74520979, 0.44542557,
        0.45629049],
       [0.37686077, 0.66715926, 0.54707224, ..., 0.60623628, 0.62869843,
        0.27435234],
       [0.23917198, 0.19179647, 0.47591831, ..., 0.14611379, 0.66710099,
        0.24164405],
       ...,
       [0.51716536, 0.7090202 , 0.5411244 , ..., 0.49518522, 0.2609199 ,
        0.25505157],
       [0.60474278, 0.59479706, 0.6909511 , ..., 0.31743146, 0.10972517,
        0.58257973],
       [0.46432524, 0.68978447, 0.48123651, ..., 0.68261767, 0.38838552,
        0.47128785]])

Convert to dataframe

In [16]:
import dask.array as da

In [ ]:
columns=list(data_train.columns.values)
columns

['var_0',
 'var_1',
 'var_2',
 'var_3',
 'var_4',
 'var_5',
 'var_6',
 'var_7',
 'var_8',
 'var_9',
 'var_10',
 'var_11',
 'var_12',
 'var_13',
 'var_14',
 'var_15',
 'var_16',
 'var_17',
 'var_18',
 'var_19',
 'var_20',
 'var_21',
 'var_22',
 'var_23',
 'var_24',
 'var_25',
 'var_26',
 'var_27',
 'var_28',
 'var_29',
 'var_30',
 'var_31',
 'var_32',
 'var_33',
 'var_34',
 'var_35',
 'var_36',
 'var_37',
 'var_38',
 'var_39',
 'var_40',
 'var_41',
 'var_42',
 'var_43',
 'var_44',
 'var_45',
 'var_46',
 'var_47',
 'var_48',
 'var_49',
 'var_50',
 'var_51',
 'var_52',
 'var_53',
 'var_54',
 'var_55',
 'var_56',
 'var_57',
 'var_58',
 'var_59',
 'var_60',
 'var_61',
 'var_62',
 'var_63',
 'var_64',
 'var_65',
 'var_66',
 'var_67',
 'var_68',
 'var_69',
 'var_70',
 'var_71',
 'var_72',
 'var_73',
 'var_74',
 'var_75',
 'var_76',
 'var_77',
 'var_78',
 'var_79',
 'var_80',
 'var_81',
 'var_82',
 'var_83',
 'var_84',
 'var_85',
 'var_86',
 'var_87',
 'var_88',
 'var_89',
 'var_90',
 'var_91'

In [ ]:
train_ss = da.ones(train_ss,chunks=(20,20))

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.86 GB -- Worker memory limit: 4.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.98 GB -- Worker memory limit: 4.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.95 GB -- Worker memory limit: 4.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.97 GB -- Worker memory limit: 4.00 GB
distributed.worker - WARN

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.97 GB -- Worker memory limit: 4.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.97 GB -- Worker memory limit: 4.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.97 GB -- Worker memory limit: 4.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.97 GB -- Worker memory limit: 4.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.97 GB -- Worker memory limit: 4.00 GB
distributed.worker - WARN

In [ ]:
train_ss = dd.io.from_dask_array(train_ss,columns)
train_ss.head()

In [ ]:
train_ss = list(data_train.columns.values)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
%time train_mms = MinMaxScaler().fit_transform(data_train)

In [ ]:
train_mms

In [ ]:
%time test_mms = MinMaxScaler().fit_transform(data_test)

In [ ]:
test_mms

The data still have negatives numbers so just skip and continue to create machine learning model and see what accuracy score value

Convert to data frame